In [ ]:
import yaml
from peft import LoraConfig, get_peft_model
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling)
from trl import SFTTrainer
from components.instructor import Instructor

import torch
torch.manual_seed(42)

# Load config file into dictionary
with open('./config/instructor_config.yaml', 'r') as file:
    config = yaml.safe_load(file)
print(config)

instructor = Instructor(config)
instructor.load_model()
instructor.load_data(test_size=0.005)
# instructor.train_dataset['text']

In [24]:
# Load config file into dictionary -- ONLY CHANGE TRAINING AND LORA PARAMETERS HERE
with open('./config/instructor_config.yaml', 'r') as file:
    config = yaml.safe_load(file)
print(config)

# data_collator = DataCollatorForLanguageModeling(tokenizer=instructor.tokenizer, mlm=False)

# LoRA configuration
peft_config = LoraConfig(**config["lora_params"])

lora_model = get_peft_model(instructor.model, peft_config)
lora_model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(**config["training_params"])

# Create trainer
trainer = SFTTrainer(
    model=instructor.model,
    train_dataset=instructor.train_dataset,
    eval_dataset=instructor.test_dataset,
    peft_config=peft_config,
    # data_collator=data_collator,
    dataset_text_field="text",
    max_seq_length=config["max_length"],
    tokenizer=instructor.tokenizer,
    args=training_args,
    packing=config["packing"]
)

trainer.train()

KeyboardInterrupt: 